In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [21]:
small_set_data = pd.read_parquet("data/Zzzs_train.parquet")

In [22]:
print(small_set_data)

             series_id    step                 timestamp     anglez    enmo  \
0         08db4255286f       0  2018-11-05T10:00:00-0400 -30.845301  0.0447   
1         08db4255286f       1  2018-11-05T10:00:05-0400 -34.181801  0.0443   
2         08db4255286f       2  2018-11-05T10:00:10-0400 -33.877102  0.0483   
3         08db4255286f       3  2018-11-05T10:00:15-0400 -34.282101  0.0680   
4         08db4255286f       4  2018-11-05T10:00:20-0400 -34.385799  0.0768   
...                ...     ...                       ...        ...     ...   
13165555  d5e47b94477e  395815  2017-12-02T15:44:35-0500 -10.064100  0.1552   
13165556  d5e47b94477e  395816  2017-12-02T15:44:40-0500 -10.006700  0.1139   
13165557  d5e47b94477e  395817  2017-12-02T15:44:45-0500  -3.328600  0.1245   
13165558  d5e47b94477e  395818  2017-12-02T15:44:50-0500  -5.903100  0.1363   
13165559  d5e47b94477e  395819  2017-12-02T15:44:55-0500 -11.691100  0.0995   

          awake  
0             1  
1             1

In [23]:
data_series = {}
for series in set(small_set_data["series_id"]):
    data_series[series] = small_set_data[small_set_data["series_id"] == series]

In [24]:
# Function to convert timestamp string to sinusoidal wave representation
def convert_to_sine_wave(timestamp_str):
    timestamp_dt = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S%z")
    time_in_hours = timestamp_dt.hour + timestamp_dt.minute / 60  # Convert minute to fraction of an hour
    return np.sin(2 * np.pi * time_in_hours / 24)  # 24 hours is the period


In [25]:
# Function to convert timestamp string to numerical value
def convert_to_numerical(timestamp_str):
    timestamp_dt = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S%z")
    return timestamp_dt.hour * 100 + timestamp_dt.minute

In [26]:
# Iterate through the dictionary and modify each DataFrame
for key, df in data_series.items():
    df_copy = df.copy()
    # Apply the custom function to the specified feature and create a new column
    df_copy['sinusoidal_time'] = df_copy['timestamp'].apply(convert_to_sine_wave)
    df_copy['numerical_time'] = df_copy['timestamp'].apply(convert_to_numerical)
    data_series[key] = df_copy

Trying LSTM + Attention

In [39]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Attention, Concatenate
from sklearn.utils import class_weight

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

# Assuming your dictionary is named 'data_dict'
all_series_ids = list(data_series.keys())

# Split series into training and validation sets
train_series_ids, val_series_ids = train_test_split(all_series_ids, test_size=0.2, random_state=42)

#Definition of scaler
scaler = MinMaxScaler()

# Select relevant columns
input_cols = ["anglez", "enmo", "numerical_time", "sinusoidal_time"]
label_col = "awake"

# Load and preprocess training data
train_data = []
for series_id in train_series_ids:
    df = data_series[series_id]
    X = df[input_cols].values
    y = df[label_col].values
    X_normalized = scaler.fit_transform(X)
    X_reshaped = X_normalized.reshape((X_normalized.shape[0], 1, X_normalized.shape[1]))
    train_data.append((X_reshaped, y))

# Concatenate all training data
X_train = np.concatenate([data[0] for data in train_data], axis=0)
y_train = np.concatenate([data[1] for data in train_data], axis=0)



#y_train = y_train.reshape((y_train.shape[0],1,1))

# Calculate class weights to handle imbalance
#classes = np.unique(y_train)
#class_weights = class_weight.compute_class_weight('balanced', classes=classes, y=y_train)
#class_weights_dict = dict(enumerate(class_weights))
# Repeat class weights for each time step
#class_weights_per_timestep = np.tile(class_weights, (1, 1))

y_train = y_train.reshape((y_train.shape[0],1,1))

# Load and preprocess validation data
val_data = []
for series_id in val_series_ids:
    df = data_series[series_id]
    X = df[input_cols].values
    y = df[label_col].values
    X_normalized = scaler.transform(X)  # Use the same scaler as training data
    X_reshaped = X_normalized.reshape((X_normalized.shape[0], 1, X_normalized.shape[1]))
    val_data.append((X_reshaped, y))

# Concatenate all validation data
X_val = np.concatenate([data[0] for data in val_data], axis=0)
y_val = np.concatenate([data[1] for data in val_data], axis=0)
y_val = y_val.reshape((y_val.shape[0],1,1))


# Build LSTM model with attention mechanism
input_layer = Input(shape=(1, len(input_cols)))
lstm_layer = LSTM(256, return_sequences=True)(input_layer)
attention_layer = Attention()([lstm_layer, lstm_layer])
attention_output = Concatenate(axis=-1)([lstm_layer, attention_layer])
output_layer = Dense(1, activation='sigmoid')(attention_output)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



# Define early stopping and TensorBoard callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

# Define a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath='model/non_window_allseries_sin_num_LSTM256_W_attention.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch',
    options=None  # Set options explicitly to None
)



2023-11-09 18:04:57.813746: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-09 18:04:57.815265: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-09 18:04:57.816453: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [43]:

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping, tensorboard, checkpoint])

#model.fit(train_dataset, epochs=20, validation_data=val_dataset, class_weight=class_weights_dict, callbacks=[early_stopping, tensorboard, checkpoint])






Epoch 1/20
320039/320040 [============================>.] - ETA: 0s - loss: 0.2081 - accuracy: 0.9153
Epoch 1: val_loss did not improve from 0.25239
320040/320040 [==============================] - 2113s 7ms/step - loss: 0.2081 - accuracy: 0.9153 - val_loss: 0.2555 - val_accuracy: 0.8973
Epoch 2/20
320040/320040 [==============================] - ETA: 0s - loss: 0.2072 - accuracy: 0.9157
Epoch 2: val_loss did not improve from 0.25239
320040/320040 [==============================] - 2110s 7ms/step - loss: 0.2072 - accuracy: 0.9157 - val_loss: 0.2557 - val_accuracy: 0.8991
Epoch 3/20
320039/320040 [============================>.] - ETA: 0s - loss: 0.2063 - accuracy: 0.9161
Epoch 3: val_loss did not improve from 0.25239
320040/320040 [==============================] - 2100s 7ms/step - loss: 0.2063 - accuracy: 0.9161 - val_loss: 0.2603 - val_accuracy: 0.8971
Epoch 4/20
320034/320040 [============================>.] - ETA: 0s - loss: 0.2055 - accuracy: 0.9164
Epoch 4: val_loss did not impro

NameError: name 'Y_val' is not defined

In [44]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

91384/91384 [==============================] - 203s 2ms/step - loss: 0.2555 - accuracy: 0.8973
Test Loss: 0.25545960664749146, Test Accuracy: 0.8973230719566345


Trying Bi-directional LSTM + Attention

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Attention, Concatenate
from sklearn.utils import class_weight

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Bidirectional

# Assuming your dictionary is named 'data_dict'
all_series_ids = list(data_series.keys())

# Split series into training and validation sets
train_series_ids, val_series_ids = train_test_split(all_series_ids, test_size=0.2, random_state=42)

#Definition of scaler
scaler = MinMaxScaler()

# Select relevant columns
input_cols = ["anglez", "enmo", "numerical_time", "sinusoidal_time"]
label_col = "awake"

# Load and preprocess training data
train_data = []
for series_id in train_series_ids:
    df = data_series[series_id]
    X = df[input_cols].values
    y = df[label_col].values
    X_normalized = scaler.fit_transform(X)
    X_reshaped = X_normalized.reshape((X_normalized.shape[0], 1, X_normalized.shape[1]))
    train_data.append((X_reshaped, y))

# Concatenate all training data
X_train = np.concatenate([data[0] for data in train_data], axis=0)
y_train = np.concatenate([data[1] for data in train_data], axis=0)



#y_train = y_train.reshape((y_train.shape[0],1,1))

# Calculate class weights to handle imbalance
#classes = np.unique(y_train)
#class_weights = class_weight.compute_class_weight('balanced', classes=classes, y=y_train)
#class_weights_dict = dict(enumerate(class_weights))
# Repeat class weights for each time step
#class_weights_per_timestep = np.tile(class_weights, (1, 1))

y_train = y_train.reshape((y_train.shape[0],1,1))

# Load and preprocess validation data
val_data = []
for series_id in val_series_ids:
    df = data_series[series_id]
    X = df[input_cols].values
    y = df[label_col].values
    X_normalized = scaler.transform(X)  # Use the same scaler as training data
    X_reshaped = X_normalized.reshape((X_normalized.shape[0], 1, X_normalized.shape[1]))
    val_data.append((X_reshaped, y))

# Concatenate all validation data
X_val = np.concatenate([data[0] for data in val_data], axis=0)
y_val = np.concatenate([data[1] for data in val_data], axis=0)
y_val = y_val.reshape((y_val.shape[0],1,1))


# Build LSTM model with bidirectional LSTM and attention mechanism
input_layer = Input(shape=(1, len(input_cols)))
bidirectional_lstm_layer = Bidirectional(LSTM(256, return_sequences=True))(input_layer)
attention_forward = Attention()([bidirectional_lstm_layer, bidirectional_lstm_layer])
attention_backward = Attention()([bidirectional_lstm_layer, bidirectional_lstm_layer])
attention_output = Concatenate(axis=-1)([bidirectional_lstm_layer, attention_forward, attention_backward])
output_layer = Dense(1, activation='sigmoid')(attention_output)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])




# Define early stopping and TensorBoard callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

# Define a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath='model/non_window_allseries_sin_num_BiDrectional_LSTM256_W_attention.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch',
    options=None  # Set options explicitly to None
)



2023-11-11 12:42:15.399506: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-11 12:42:15.402243: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-11 12:42:15.404109: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=128, validation_data=(X_val, y_val), callbacks=[early_stopping, tensorboard, checkpoint])


# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')




In [11]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


68367/68367 [==============================] - 358s 5ms/step - loss: 0.2092 - accuracy: 0.9165
Test Loss: 0.20918025076389313, Test Accuracy: 0.9165267944335938


no time feature bi-directional

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Attention, Concatenate
from sklearn.utils import class_weight

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Bidirectional

# Assuming your dictionary is named 'data_dict'
all_series_ids = list(data_series.keys())

# Split series into training and validation sets
train_series_ids, val_series_ids = train_test_split(all_series_ids, test_size=0.2, random_state=42)

#Definition of scaler
scaler = MinMaxScaler()

# Select relevant columns
input_cols = ["anglez", "enmo"]#, "numerical_time", "sinusoidal_time"]
label_col = "awake"

# Load and preprocess training data
train_data = []
for series_id in train_series_ids:
    df = data_series[series_id]
    X = df[input_cols].values
    y = df[label_col].values
    X_normalized = scaler.fit_transform(X)
    X_reshaped = X_normalized.reshape((X_normalized.shape[0], 1, X_normalized.shape[1]))
    train_data.append((X_reshaped, y))

# Concatenate all training data
X_train = np.concatenate([data[0] for data in train_data], axis=0)
y_train = np.concatenate([data[1] for data in train_data], axis=0)


y_train = y_train.reshape((y_train.shape[0],1,1))

# Load and preprocess validation data
val_data = []
for series_id in val_series_ids:
    df = data_series[series_id]
    X = df[input_cols].values
    y = df[label_col].values
    X_normalized = scaler.transform(X)  # Use the same scaler as training data
    X_reshaped = X_normalized.reshape((X_normalized.shape[0], 1, X_normalized.shape[1]))
    val_data.append((X_reshaped, y))

# Concatenate all validation data
X_val = np.concatenate([data[0] for data in val_data], axis=0)
y_val = np.concatenate([data[1] for data in val_data], axis=0)
y_val = y_val.reshape((y_val.shape[0],1,1))


# Build LSTM model with bidirectional LSTM and attention mechanism
input_layer = Input(shape=(1, len(input_cols)))
bidirectional_lstm_layer = Bidirectional(LSTM(256, return_sequences=True))(input_layer)
attention_forward = Attention()([bidirectional_lstm_layer, bidirectional_lstm_layer])
attention_backward = Attention()([bidirectional_lstm_layer, bidirectional_lstm_layer])
attention_output = Concatenate(axis=-1)([bidirectional_lstm_layer, attention_forward, attention_backward])
output_layer = Dense(1, activation='sigmoid')(attention_output)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])




# Define early stopping and TensorBoard callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

# Define a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath='model/non_window_allseries_no_time_BiDrectional_LSTM256_W_attention.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch',
    options=None  # Set options explicitly to None
)



2023-11-11 17:38:36.924674: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-11 17:38:36.926572: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-11 17:38:36.928400: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [15]:

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=128, validation_data=(X_val, y_val), callbacks=[early_stopping, tensorboard, checkpoint])


# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')




Epoch 1/20


2023-11-11 17:38:44.017025: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-11 17:38:44.020053: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-11 17:38:44.022719: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

85764/85765 [============================>.] - ETA: 0s - loss: 0.4789 - accuracy: 0.7627

2023-11-11 18:44:42.177408: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-11 18:44:42.180587: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-11 18:44:42.183556: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_loss improved from inf to 0.49969, saving model to model/non_window_allseries_no_time_BiDrectional_LSTM256_W_attention.h5
85765/85765 [==============================] - 4194s 49ms/step - loss: 0.4789 - accuracy: 0.7627 - val_loss: 0.4997 - val_accuracy: 0.7237
Epoch 2/20
85765/85765 [==============================] - ETA: 0s - loss: 0.4686 - accuracy: 0.7675
Epoch 2: val_loss improved from 0.49969 to 0.49330, saving model to model/non_window_allseries_no_time_BiDrectional_LSTM256_W_attention.h5
85765/85765 [==============================] - 4180s 49ms/step - loss: 0.4686 - accuracy: 0.7675 - val_loss: 0.4933 - val_accuracy: 0.7434
Epoch 3/20
85765/85765 [==============================] - ETA: 0s - loss: 0.4674 - accuracy: 0.7693
Epoch 3: val_loss improved from 0.49330 to 0.49269, saving model to model/non_window_allseries_no_time_BiDrectional_LSTM256_W_attention.h5
85765/85765 [==============================] - 4175s 49ms/step - loss: 0.4674 - accuracy: 0.7693 - val_loss:

tried stacking two Bidirect-LSTM attention blocks together, took way too long to train, accuracy seems to get stuck at ~90% anyways

In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Attention, Concatenate
from sklearn.utils import class_weight

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Bidirectional

# Assuming your dictionary is named 'data_dict'
all_series_ids = list(data_series.keys())

# Split series into training and validation sets
train_series_ids, val_series_ids = train_test_split(all_series_ids, test_size=0.2, random_state=42)

#Definition of scaler
scaler = MinMaxScaler()

# Select relevant columns
input_cols = ["anglez", "enmo", "numerical_time", "sinusoidal_time"]
label_col = "awake"

# Load and preprocess training data
train_data = []
for series_id in train_series_ids:
    df = data_series[series_id]
    X = df[input_cols].values
    y = df[label_col].values
    X_normalized = scaler.fit_transform(X)
    X_reshaped = X_normalized.reshape((X_normalized.shape[0], 1, X_normalized.shape[1]))
    train_data.append((X_reshaped, y))

# Concatenate all training data
X_train = np.concatenate([data[0] for data in train_data], axis=0)
y_train = np.concatenate([data[1] for data in train_data], axis=0)



#y_train = y_train.reshape((y_train.shape[0],1,1))

# Calculate class weights to handle imbalance
#classes = np.unique(y_train)
#class_weights = class_weight.compute_class_weight('balanced', classes=classes, y=y_train)
#class_weights_dict = dict(enumerate(class_weights))
# Repeat class weights for each time step
#class_weights_per_timestep = np.tile(class_weights, (1, 1))

y_train = y_train.reshape((y_train.shape[0],1,1))

# Load and preprocess validation data
val_data = []
for series_id in val_series_ids:
    df = data_series[series_id]
    X = df[input_cols].values
    y = df[label_col].values
    X_normalized = scaler.transform(X)  # Use the same scaler as training data
    X_reshaped = X_normalized.reshape((X_normalized.shape[0], 1, X_normalized.shape[1]))
    val_data.append((X_reshaped, y))

# Concatenate all validation data
X_val = np.concatenate([data[0] for data in val_data], axis=0)
y_val = np.concatenate([data[1] for data in val_data], axis=0)
y_val = y_val.reshape((y_val.shape[0],1,1))

'''
# Build LSTM model with bidirectional LSTM and attention mechanism
input_layer = Input(shape=(1, len(input_cols)))
bidirectional_lstm_layer = Bidirectional(LSTM(256, return_sequences=True))(input_layer)
attention_forward = Attention()([bidirectional_lstm_layer, bidirectional_lstm_layer])
attention_backward = Attention()([bidirectional_lstm_layer, bidirectional_lstm_layer])
attention_output = Concatenate(axis=-1)([bidirectional_lstm_layer, attention_forward, attention_backward])
output_layer = Dense(1, activation='sigmoid')(attention_output)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

'''

# Build LSTM model with multiple bidirectional LSTM and attention blocks
input_layer = Input(shape=(1, len(input_cols)))

# First Bidirectional LSTM and Attention block
bidirectional_lstm_layer_1 = Bidirectional(LSTM(256, return_sequences=True))(input_layer)
attention_forward_1 = Attention()([bidirectional_lstm_layer_1, bidirectional_lstm_layer_1])
attention_backward_1 = Attention()([bidirectional_lstm_layer_1, bidirectional_lstm_layer_1])
attention_output_1 = Concatenate(axis=-1)([bidirectional_lstm_layer_1, attention_forward_1, attention_backward_1])

# Second Bidirectional LSTM and Attention block
bidirectional_lstm_layer_2 = Bidirectional(LSTM(256, return_sequences=True))(attention_output_1)
attention_forward_2 = Attention()([bidirectional_lstm_layer_2, bidirectional_lstm_layer_2])
attention_backward_2 = Attention()([bidirectional_lstm_layer_2, bidirectional_lstm_layer_2])
attention_output_2 = Concatenate(axis=-1)([bidirectional_lstm_layer_2, attention_forward_2, attention_backward_2])

# Dense output layer
output_layer = Dense(1, activation='sigmoid')(attention_output_2)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



# Define early stopping and TensorBoard callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

# Define a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath='model/non_window_allseries_sin_num_stacked_2_BiDrectional_LSTM256_W_attention.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch',
    options=None  # Set options explicitly to None
)



2023-11-12 10:08:40.017240: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-12 10:08:40.019170: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-12 10:08:40.020995: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=128, validation_data=(X_val, y_val), callbacks=[early_stopping, tensorboard, checkpoint])


# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')




Epoch 1/20


2023-11-12 10:08:42.373214: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-12 10:08:42.376091: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-12 10:08:42.378724: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

 9491/85765 [==>...........................] - ETA: 5:51:36 - loss: 0.2289 - accuracy: 0.9061

going back to one bi-directional attention block but with a different hidden layer size

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Attention, Concatenate
from sklearn.utils import class_weight

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Bidirectional

# Assuming your dictionary is named 'data_dict'
all_series_ids = list(data_series.keys())

# Split series into training and validation sets
train_series_ids, val_series_ids = train_test_split(all_series_ids, test_size=0.2, random_state=42)

#Definition of scaler
scaler = MinMaxScaler()

# Select relevant columns
input_cols = ["anglez", "enmo", "numerical_time", "sinusoidal_time"]
label_col = "awake"

# Load and preprocess training data
train_data = []
for series_id in train_series_ids:
    df = data_series[series_id]
    X = df[input_cols].values
    y = df[label_col].values
    X_normalized = scaler.fit_transform(X)
    X_reshaped = X_normalized.reshape((X_normalized.shape[0], 1, X_normalized.shape[1]))
    train_data.append((X_reshaped, y))

# Concatenate all training data
X_train = np.concatenate([data[0] for data in train_data], axis=0)
y_train = np.concatenate([data[1] for data in train_data], axis=0)



#y_train = y_train.reshape((y_train.shape[0],1,1))

# Calculate class weights to handle imbalance
#classes = np.unique(y_train)
#class_weights = class_weight.compute_class_weight('balanced', classes=classes, y=y_train)
#class_weights_dict = dict(enumerate(class_weights))
# Repeat class weights for each time step
#class_weights_per_timestep = np.tile(class_weights, (1, 1))

y_train = y_train.reshape((y_train.shape[0],1,1))

# Load and preprocess validation data
val_data = []
for series_id in val_series_ids:
    df = data_series[series_id]
    X = df[input_cols].values
    y = df[label_col].values
    X_normalized = scaler.transform(X)  # Use the same scaler as training data
    X_reshaped = X_normalized.reshape((X_normalized.shape[0], 1, X_normalized.shape[1]))
    val_data.append((X_reshaped, y))

# Concatenate all validation data
X_val = np.concatenate([data[0] for data in val_data], axis=0)
y_val = np.concatenate([data[1] for data in val_data], axis=0)
y_val = y_val.reshape((y_val.shape[0],1,1))


# Build LSTM model with bidirectional LSTM and attention mechanism
input_layer = Input(shape=(1, len(input_cols)))
bidirectional_lstm_layer = Bidirectional(LSTM(256, return_sequences=True))(input_layer)

# Accessing forward and backward outputs separately
forward_lstm_output = bidirectional_lstm_layer[:, :, :256]  # Assuming 256 units
backward_lstm_output = bidirectional_lstm_layer[:, :, 256:]


attention_forward = Attention()([forward_lstm_output, forward_lstm_output])
attention_backward = Attention()([backward_lstm_output, backward_lstm_output])
attention_output = Concatenate(axis=-1)([bidirectional_lstm_layer, attention_forward, attention_backward])
output_layer = Dense(1, activation='sigmoid')(attention_output)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])




# Define early stopping and TensorBoard callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

# Define a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath='model/non_window_allseries_sin_num_seperate_attn_BiDrectional_LSTM256_W_attention.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch',
    options=None  # Set options explicitly to None
)



2023-11-12 13:44:38.144349: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-12 13:44:38.147142: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-12 13:44:38.156447: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [28]:

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=128, validation_data=(X_val, y_val), callbacks=[early_stopping, tensorboard, checkpoint])


# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')




2023-11-12 13:46:32.368551: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 171141120 exceeds 10% of free system memory.
2023-11-12 13:46:32.445090: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 85570560 exceeds 10% of free system memory.


Epoch 1/20


2023-11-12 13:46:33.673838: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 85570560 exceeds 10% of free system memory.
2023-11-12 13:46:33.712965: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 85570560 exceeds 10% of free system memory.
2023-11-12 13:46:34.536703: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-12 13:46:34.538474: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{

83565/83565 [==============================] - ETA: 0s - loss: 0.2210 - accuracy: 0.9097

2023-11-12 14:28:51.645558: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-12 14:28:51.648010: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-12 14:28:51.650349: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_loss improved from inf to 0.22480, saving model to model/non_window_allseries_sin_num_seperate_attn_BiDrectional_LSTM256_W_attention.h5
83565/83565 [==============================] - 2733s 33ms/step - loss: 0.2210 - accuracy: 0.9097 - val_loss: 0.2248 - val_accuracy: 0.9117
Epoch 2/20
83565/83565 [==============================] - ETA: 0s - loss: 0.2145 - accuracy: 0.9120
Epoch 2: val_loss did not improve from 0.22480
83565/83565 [==============================] - 2748s 33ms/step - loss: 0.2145 - accuracy: 0.9120 - val_loss: 0.2354 - val_accuracy: 0.9072
Epoch 3/20
83564/83565 [============================>.] - ETA: 0s - loss: 0.2134 - accuracy: 0.9125
Epoch 3: val_loss did not improve from 0.22480
83565/83565 [==============================] - 2921s 35ms/step - loss: 0.2134 - accuracy: 0.9125 - val_loss: 0.2323 - val_accuracy: 0.9098
Epoch 4/20
83565/83565 [==============================] - ETA: 0s - loss: 0.2128 - accuracy: 0.9127
Epoch 4: val_loss did not improve from 